<a href="https://colab.research.google.com/github/siddharath28/Excel-Sales-Analytics/blob/main/process1_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os

def process_gps_data(input_file, output_dir):

    df = pd.read_parquet(input_file)


    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')


    df = df.sort_values(by=['unit', 'timestamp'])


    for unit, unit_data in df.groupby('unit'):
        current_trip_start = unit_data.iloc[0]['timestamp']
        current_trip_number = 1


        for i in range(1, len(unit_data)):
            time_diff = unit_data.iloc[i]['timestamp'] - unit_data.iloc[i - 1]['timestamp']


            if time_diff > timedelta(hours=7):

                trip_file_name = f"{unit}_trip_{current_trip_number}.csv"
                trip_file_path = os.path.join(output_dir, trip_file_name)


                unit_data.iloc[i - 1].to_frame().T.to_csv(trip_file_path, index=False)


                current_trip_start = unit_data.iloc[i]['timestamp']
                current_trip_number += 1


        trip_file_name = f"{unit}_trip_{current_trip_number}.csv"
        trip_file_path = os.path.join(output_dir, trip_file_name)
        unit_data.iloc[-1].to_frame().T.to_csv(trip_file_path, index=False)

if __name__ == "__main__":
    # Example usage:
    input_file = "raw_data.parquet"
    output_directory = "output_trips"


    os.makedirs(output_directory, exist_ok=True)

    process_gps_data(input_file, output_directory)
